In [7]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datestart='2025-01-02'
dateend='2024--01'
datefilter=''

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
df=pd.read_sql_query('Select Distinct * from allmatches',con=devengine)
# Ensure Winner_Sets and Loser_Sets are integers
df["Winner_Sets"] = df["Winner_Sets"].astype(int)
df["Loser_Sets"] = df["Loser_Sets"].astype(int)

# Calculate the total number of completed sets
df["Completed_Sets"] = df["Winner_Sets"] + df["Loser_Sets"]

# Populate Set Winner columns based on set scores and Completed_Sets
for set_num in range(1, 6):
    set_winner_col = f"Set {set_num} Winner"
    w_col = f"w{set_num}"
    l_col = f"l{set_num}"
    
    # Ensure the column exists
    if w_col in df.columns and l_col in df.columns:
        # Determine the winner for the set
        df[set_winner_col] = df.apply(
            lambda row: row["Player_1"] if row[w_col] > row[l_col] else row["Player_2"], axis=1
        )
    else:
        df[set_winner_col] = None  # Initialize with None if scores don't exist

    # Clear values for sets beyond the number of completed sets
    df[set_winner_col] = df.apply(
        lambda row: row[set_winner_col] if set_num <= row["Completed_Sets"] else None, axis=1
    )
df["fav_odds"] = df.apply(
    lambda row: min(row["Player_1_Odds"], row["Player_2_Odds"]), axis=1
)
df["dog_odds"] = df.apply(
    lambda row: max(row["Player_1_Odds"], row["Player_2_Odds"]), axis=1
)
# Create columns for Fav and Dog based on odds
df["Fav"] = df.apply(
    lambda row: row["Player_1"] if row["Player_1_Odds"] == row["fav_odds"] else row["Player_2"], axis=1
)
df["Dog"] = df.apply(
    lambda row: row["Player_2"] if row["Player_1_Odds"] == row["fav_odds"] else row["Player_1"], axis=1
)

# Adjust the Set Winner columns to reflect the favorite and underdog
for set_num in range(1, 6):
    set_winner_col = f"Set {set_num} Winner"
    
    # Reorganize the Set Winner columns based on the favorite and underdog
    df[set_winner_col] = df.apply(
        lambda row: row["Fav"] if row[set_winner_col] == row["Player_1"] else row["Dog"], axis=1
    )

# Ensure Winner_Sets and Loser_Sets are integers
df["Winner_Sets"] = df["Winner_Sets"].astype(int)
df["Loser_Sets"] = df["Loser_Sets"].astype(int)

# Calculate the total number of completed sets
df["Completed_Sets"] = df["Winner_Sets"] + df["Loser_Sets"]


df["Match_Type"] = df.apply(
    lambda row: "Best of 5" if row["Winner_Sets"] + row["Loser_Sets"] > 3 else "Best of 3", axis=1
)
# Loop through Set 1 Winner to Set 5 Winner columns
for set_num in range(1, 6):
    set_winner_col = f"Set {set_num} Winner"
    set_winner_fav_col = f"Set {set_num} Winner_Fav"
    
    # Check if the necessary columns are present in the DataFrame
    if "Fav" in df.columns and "Dog" in df.columns:
        # Create new columns Set X Winner_Fav based on the logic
        df[set_winner_fav_col] = df.apply(
            lambda row: 'Fav' if row[set_winner_col] == row["Fav"] else 'Dog', axis=1
        )
    else:
        df[set_winner_fav_col] = None  # Initialize with None if "Fav" or "Dog" columns don't exist

#df=df[df['Set 2 Winner'].notna()]
#df=df[df['Set 1 Winner_Fav']=='Dog']
#df=df[df['Set 3 Winner_Fav']=='Dog']
#df[['Date','Set 1 Winner_Fav','Set 1 Winner','w1','l1','Set 2 Winner','Set 2 Winner_Fav','w2','l2','Set 3 Winner','Set 3 Winner_Fav','w3','l3','Fav','Dog','fav_odds']][df['w3']!=" "]

In [8]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datestart='2025-01-02'
dateend='2024--01'
datefilter=''

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
df=pd.read_sql_query('Select Distinct * from allmatches',con=devengine)
# Ensure Winner_Sets and Loser_Sets are integers

# Example DataFrame: df
set_cols = [('w1', 'l1'), ('w2', 'l2'), ('w3', 'l3'), ('w4', 'l4'), ('w5', 'l5')]

# Add columns to record set winners
for i, (w_col, l_col) in enumerate(set_cols, start=1):
    def determine_set_winner(row):
        w_games = row[w_col]
        l_games = row[l_col]
        if pd.isna(w_games) or pd.isna(l_games):
            return np.nan  # Set not played
        if w_games > l_games:
            return row['Player_1']
        elif w_games < l_games:
            return row['Player_2']
        else:
            return 'Tie'  # Rare, but possible in corrupted data

    df[f'set_{i}_winner'] = df.apply(determine_set_winner, axis=1)

clean=df[['Player_1','Player_2','Player_1_Odds','Player_2_Odds','set_1_winner','set_2_winner','set_3_winner','Surface']]
clean=clean[clean['set_1_winner']!='Tie']
data=clean[clean['set_2_winner']!='Tie'].copy()


data['Set12']=data['set_1_winner']==data['set_2_winner']

def get_fav_dog(row):
    if row['Player_1_Odds'] < row['Player_2_Odds']:
        return pd.Series([row['Player_1_Odds'], row['Player_2_Odds'], row['Player_1'], row['Player_2']],
                         index=['fav_odds', 'dog_odds', 'fav', 'dog'])
    else:
        return pd.Series([row['Player_2_Odds'], row['Player_1_Odds'], row['Player_2'], row['Player_1']],
                         index=['fav_odds', 'dog_odds', 'fav', 'dog'])

# Apply to your dataframe
data[['fav_odds', 'dog_odds', 'fav', 'dog']] = data.apply(get_fav_dog, axis=1)




In [9]:
fav_1stwin=data.copy()
fav_1stwin['Set1_fav']=fav_1stwin['set_1_winner']==fav_1stwin['fav']
fav_1stwin=fav_1stwin[fav_1stwin['Set1_fav']==True]
fav_1stwin=fav_1stwin[fav_1stwin['fav_odds']>1]
fav_1stwin=fav_1stwin[fav_1stwin['fav_odds']<16]
fav_1stwin=fav_1stwin[fav_1stwin['Surface']=='Clay']
results = fav_1stwin.groupby('fav').agg(
    Games_Played=('Set12', 'count'),
    SecondSet_Percentage=('Set12', 'mean')
).reset_index()
results=results[results['Games_Played']>10]
results=results[results['SecondSet_Percentage']>.8]
results.sort_values(by=['SecondSet_Percentage','Games_Played'],ascending=False)
lowodds=pd.read_pickle(r'.\Clay_Today')
merge=lowodds.merge(results,left_on='Fav',right_on='fav')
print('Second set selections Favs (if they won 1st set)')
merge[merge['Fav_Odds']>1][['Time','Fav_Odds','Fav','Dog','SecondSet_Percentage']]

Second set selections Favs (if they won 1st set)


,Time,Fav_Odds,Fav,Dog,SecondSet_Percentage
0,01:05,1.64,Valentin Vacherot,Alex Molcan,0.851852
1,03:50,1.66,Gonzalo Villanueva,Lucas Gerch,0.846154
2,21:30,1.62,Astra Sharma,Veronika Erjavec,0.846154
3,22:00,1.08,Sara Bejlek,Carlota Martinez Cirez,0.809524
4,23:30,1.43,Oksana Selekhmeteva,Hanne Vandewinkel,0.866667


In [10]:
fav_1stloss=data.copy()
fav_1stloss['Set1_fav']=fav_1stloss['set_1_winner']==fav_1stloss['fav']
fav_1stloss=fav_1stloss[fav_1stloss['Set1_fav']==False]
fav_1stloss=fav_1stloss[fav_1stloss['fav_odds']>1]
fav_1stloss=fav_1stloss[fav_1stloss['fav_odds']<14]
fav_1stloss=fav_1stloss[fav_1stloss['Surface']=='Clay']
results = fav_1stloss.groupby('fav').agg(
    Games_Played=('Set12', 'count'),
    SecondSet_Percentage=('Set12', lambda x: 1 - x.mean())
).reset_index()
results=results[results['Games_Played']>10]
results=results[results['SecondSet_Percentage']>.50]
results.sort_values(by=['SecondSet_Percentage','Games_Played'],ascending=False)
lowodds=pd.read_pickle(r'.\Clay_Today')
merge=lowodds.merge(results,left_on='Fav',right_on='fav')
print('Second set selections Favs (if they lost first set)')
merge[merge['Fav_Odds']>1][['Time','Fav_Odds','Fav','Dog','SecondSet_Percentage']]
#fav_1stloss[fav_1stloss['fav']=='Kimmer Coppejans']

Second set selections Favs (if they lost first set)


,Time,Fav_Odds,Fav,Dog,SecondSet_Percentage
0,00:05,1.15,Lucas Reis Da Silva Joao,Igor Gimenez,0.562500
1,00:45,1.06,Cristian Garin,Abedallah Shelbayh,0.592593
2,01:05,1.64,Valentin Vacherot,Alex Molcan,0.588235
3,03:25,1.27,Francesco Passaro,Francesco Maestrelli,0.571429
4,05:55,1.28,Facundo Bagnis,Conner Huertas Del Pino,0.666667
5,07:55,1.53,Alberto Olivieri Genaro,Hernan Casanova,0.523810
6,19:00,1.12,Francesco Passaro,Viktor Durasovic,0.571429
7,21:30,1.36,Luca Van Assche,Geoffrey Blancaneaux,0.636364
8,23:00,1.24,Lourdes Carle Maria,Louisa Chirico,0.545455
9,23:00,1.30,Gauthier Onclin,Lucas Poullain,0.533333


In [11]:
fav_1stwin=data.copy()
fav_1stwin['Set1_dog']=fav_1stwin['set_1_winner']==fav_1stwin['dog']
fav_1stwin=fav_1stwin[fav_1stwin['Set1_dog']==True]
fav_1stwin=fav_1stwin[fav_1stwin['fav_odds']>1]
fav_1stwin=fav_1stwin[fav_1stwin['fav_odds']<16]
fav_1stwin=fav_1stwin[fav_1stwin['Surface']=='Clay']
results = fav_1stwin.groupby('dog').agg(
    Games_Played=('Set12', 'count'),
    SecondSet_Percentage=('Set12', 'mean')
).reset_index()
results=results[results['Games_Played']>5]
results=results[results['SecondSet_Percentage']>.75]
results.sort_values(by=['SecondSet_Percentage','Games_Played'],ascending=False)
lowodds=pd.read_pickle(r'.\Clay_Today')
merge=lowodds.merge(results,left_on='Dog',right_on='dog')
print('Second set selections Dogs (if they won first set)')
merge[merge['Fav_Odds']>1][['Time','Dog_Odds','Dog','Fav','SecondSet_Percentage']]

Second set selections Dogs (if they won first set)


,Time,Dog_Odds,Dog,Fav,SecondSet_Percentage


In [12]:
fav_1stloss=data.copy()
fav_1stloss['Set1_dog']=fav_1stloss['set_1_winner']==fav_1stloss['dog']
fav_1stloss=fav_1stloss[fav_1stloss['Set1_dog']==False]
fav_1stloss=fav_1stloss[fav_1stloss['fav_odds']>1]
fav_1stloss=fav_1stloss[fav_1stloss['fav_odds']<14]
fav_1stloss=fav_1stloss[fav_1stloss['Surface']=='Clay']
results = fav_1stloss.groupby('dog').agg(
    Games_Played=('Set12', 'count'),
    SecondSet_Percentage=('Set12', lambda x: 1 - x.mean())
).reset_index()
results=results[results['Games_Played']>10]
results=results[results['SecondSet_Percentage']>.50]
results.sort_values(by=['SecondSet_Percentage','Games_Played'],ascending=False)
lowodds=pd.read_pickle(r'.\Clay_Today')
merge=lowodds.merge(results,left_on='Dog',right_on='dog')
print('Second set selections Dogs (if they lost first set)')
merge[merge['Fav_Odds']>1][['Time','Dog_Odds','Dog','Fav','SecondSet_Percentage']]
#fav_1stloss[fav_1stloss['dog']=='Vit Kopriva']


Second set selections Dogs (if they lost first set)


,Time,Dog_Odds,Dog,Fav,SecondSet_Percentage
0,20:00,2.17,Norbert Gombos,Marko Topo,0.571429
